In [1]:
import pandas as pd
import numpy as np

In [2]:
df=pd.read_csv("twitter_sentiment.csv",encoding='ISO-8859-1')   #to prevent unicode decode error

In [3]:
df.shape

(99988, 3)

In [4]:
df.head(10)

,ItemID,Sentiment,SentimentText
0,1,0,is so sad for my APL frie...
1,2,0,I missed the New Moon trail...
2,3,1,omg its already 7:30 :O
3,4,0,.. Omgaga. Im sooo im gunna CRy. I'...
4,5,0,i think mi bf is cheating on me!!! ...
5,6,0,or i just worry too much?
6,7,1,Juuuuuuuuuuuuuuuuussssst Chillin!!
7,8,0,Sunny Again Work Tomorrow :-| ...
8,9,1,handed in my uniform today . i miss you ...
9,10,1,hmmmm.... i wonder how she my number @-)


In [5]:
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer

In [6]:
ps = PorterStemmer()
data = []

In [7]:
for i in range(0,99988):
        review=df["SentimentText"][i]
        review = re.sub('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+#]|[!*\(\),]|'\
                       '(?:%[0-9a-fA-F][0-9a-fA-F]))+','', review)
        review = re.sub("(@[A-Za-z0-9_]+)","", review)
        review = review.lower()
        review = review.split()
        review = [ps.stem(word) for word in review if not word in set(stopwords.words('english'))]
        review = ' '.join(review)
        data.append(review)

In [18]:
from sklearn.feature_extraction.text import CountVectorizer

In [46]:
cv = CountVectorizer(max_features=1000)

In [47]:
x = cv.fit_transform(data)
x = x.toarray()

In [80]:
from sklearn.externals.joblib import dump
dump(cv,"twitterdata.bin")

C:\Users\TEJU\Anaconda3\lib\site-packages\sklearn\externals\joblib\__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)


['twitterdata.bin']

In [48]:
x.shape

(99988, 1000)

In [49]:
y = df.iloc[:,1].values

In [50]:
y = y.reshape(-1, 1)

In [51]:
y.shape

(99988, 1)

In [52]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.1, random_state = 0)

In [53]:
from keras.models import Sequential
from keras.layers import Dense

In [54]:
model = Sequential()

In [55]:
model.add(Dense(activation="relu", input_dim=1000, units=150, kernel_initializer="uniform"))

In [56]:
model.add(Dense(activation="relu", units=70, kernel_initializer="uniform"))

In [57]:
model.add(Dense(activation="relu", units = 20 , kernel_initializer="uniform"))

In [58]:
model.add(Dense(activation="relu", units = 6, kernel_initializer="uniform"))

In [59]:
model.add(Dense(output_dim = 1, init = 'uniform', activation = 'sigmoid'))

C:\Users\TEJU\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="sigmoid", units=1, kernel_initializer="uniform")`
  """Entry point for launching an IPython kernel.


In [60]:
model.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

In [61]:
model.fit(x_train, y_train, batch_size = 32, epochs = 30)


Epoch 1/30
89989/89989 [==============================] - 19s 214us/step - loss: 0.5507 - accuracy: 0.7146
Epoch 2/30
89989/89989 [==============================] - 13s 145us/step - loss: 0.5129 - accuracy: 0.7462
Epoch 3/30
89989/89989 [==============================] - 13s 150us/step - loss: 0.4800 - accuracy: 0.7695
Epoch 4/30
89989/89989 [==============================] - 13s 141us/step - loss: 0.4217 - accuracy: 0.8058
Epoch 5/30
89989/89989 [==============================] - 13s 146us/step - loss: 0.3525 - accuracy: 0.8410
Epoch 6/30
89989/89989 [==============================] - 13s 142us/step - loss: 0.2922 - accuracy: 0.8686
Epoch 7/30
89989/89989 [==============================] - 13s 148us/step - loss: 0.2465 - accuracy: 0.8894
Epoch 8/30
89989/89989 [==============================] - 14s 154us/step - loss: 0.2148 - accuracy: 0.9032
Epoch 9/30
89989/89989 [==============================] - 13s 144us/step - loss: 0.1904 - accuracy: 0.9131
Epoch 10/30
89989/89989 [===========

In [62]:
model.save("twitterdata.h5")

In [90]:
y_pred = model.predict(x_test)

In [91]:
y_pred = y_pred>0.5

In [95]:
prediction = model.predict(cv.transform(["worst"]))

In [96]:
prediction = prediction>0.5

In [97]:
prediction

array([[False]])